# ResNet-152

# Hyperparameters

#### batch_size = 64
#### num_training_epochs = 200
#### lr = 0.01
#### valid_loss_stable_count = 6

#### Make learning rate lr one tenth if the number of epochs in which validation loss doesn't decrease exceeds the paramter of valid_loss_stable_count.

#### Test Loss: 0.701256
#### Test Accuracy: 71% (142/200)



In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.datasets
import torchvision.transforms as transforms
from PIL import ImageFile

import numpy as np

import torch.optim as optim
import time

In [2]:
TRAIN_TEST_CSV_PATH = '../C1-P1_Train Dev_fixed/train.csv'
TRAIN_CSV_PATH = '../C1-P1_Train Dev_fixed/train_split.csv'
VALID_CSV_PATH = '../C1-P1_Train Dev_fixed/dev.csv'

ORIGINAL_TRAIN_TEST_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Train/' 
ORIGINAL_VALID_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Dev/' 


TRAIN_DATA_PATH = '../data/train'
VALID_DATA_PATH = '../data/valid'
TEST_DATA_PATH = '../data/test'

MODEL_WEIGHTS_FILE = '../model_trained_weights/model_weights_v3_1__run_1.pt'

image_size = 224

In [3]:
# Hyperparameters

batch_size = 64
lr = 0.01
valid_loss_stable_count = 6

lr_decay_factor = 0.1
lr_lower_bound = 1e-4

num_training_epochs = 200
num_worker = 4
sgd_momentum = 0.9

In [4]:
ImageFile.LOAD_TRUNCATED_IMAGES = True


transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation(degrees=(-15, 15)),
                                transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    mean=(0.485, 0.456, 0.406),
                                    std =(0.229, 0.224, 0.225))
                               ])
transform_plain = transforms.Compose([
                            transforms.Resize(224),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize(
                                    mean=(0.485, 0.456, 0.406),
                                    std =(0.229, 0.224, 0.225))
                           ]) 

loaders_transfer = {}
data_transfer = {}

data_transfer['train'] = torchvision.datasets.ImageFolder(TRAIN_DATA_PATH, transform=transform)
loaders_transfer['train'] = torch.utils.data.DataLoader(data_transfer['train'],
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_worker)

data_transfer['valid'] = torchvision.datasets.ImageFolder(VALID_DATA_PATH, transform=transform_plain)
loaders_transfer['valid'] = torch.utils.data.DataLoader(data_transfer['valid'],
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=num_worker)
data_transfer['test'] = torchvision.datasets.ImageFolder(TEST_DATA_PATH, transform=transform_plain)
loaders_transfer['test'] = torch.utils.data.DataLoader(data_transfer['test'],
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=num_worker)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
import torchvision.models as models
import torch.nn as nn

model_transfer = models.resnet152(pretrained=True).to(device)
    
for param in model_transfer.parameters():
    param.requires_grad = False   
    
model_transfer.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 3)).to(device)
## uncomment the following line for continuing trainging only
# model_transfer.load_state_dict(torch.load(MODEL_WEIGHTS_FILE, map_location=device))

In [7]:
import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.fc.parameters(), lr = lr, momentum = 0.9)

In [8]:
def get_lr_from_optim(optimizer):
    for param_group in optimizer_transfer.param_groups:
        return param_group['lr']
    
def set_lr_to_optim(optimizer, lr):
    for param_group in optimizer_transfer.param_groups:
        param_group['lr'] = lr

In [9]:
import numpy as np
import time
import copy


# train the model
def train(n_epochs, loaders, model, optimizer, criterion, save_path):

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    # Valid Loss Stable counter
    valid_loss_stable_counter = 0
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        
        start = time.time()
        
        lr = get_lr_from_optim(optimizer)
        
        # train the model
        model.train()
        for batch_idx, (data, target) in enumerate(loaders_transfer['train']):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
            
        # validate the model

        model.eval()
        valid_corrects = 0
        for batch_idx, (data, target) in enumerate(loaders_transfer['valid']):

            data, target = data.to(device), target.to(device)
            output = model(data)
            _, preds = torch.max(output, 1)
            
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
    
            valid_corrects += torch.sum(preds == target.data)
        
        train_loss = train_loss/len(loaders_transfer['train'].dataset)
        valid_loss = valid_loss/len(loaders_transfer['valid'].dataset)
        
        epoch_acc = valid_corrects.double() / len(loaders_transfer['valid'].dataset)
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.3f} \ttime: {:.1f} \tlr={}'.format(
            epoch, 
            train_loss,
            valid_loss,
            epoch_acc,
            time.time() - start,
            lr
            ))
        
        if valid_loss < valid_loss_min:
            print('Validation loss decreased from {:.6f} to {:.6f}. Model was saved'.format(
                valid_loss_min,
                valid_loss
            ))

            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, save_path)

            valid_loss_min = valid_loss
            
            valid_loss_stable_counter = 0
        else:
            valid_loss_stable_counter += 1
            if valid_loss_stable_counter >= valid_loss_stable_count:
                valid_loss_stable_counter = 0
                lr = get_lr_from_optim(optimizer)
                lr = lr * lr_decay_factor
                if lr <= lr_lower_bound:
                    return model
                set_lr_to_optim(optimizer, lr)
    
    # return trained model
    return model

train_start = time.time()
model_transfer = train(num_training_epochs,
                       loaders_transfer,
                       model_transfer,optimizer_transfer,
                       criterion_transfer,
                       MODEL_WEIGHTS_FILE)
print("Total training time: {:.2f} seconds".format(time.time() - train_start))

Epoch: 1 	Training Loss: 0.8536 	Validation Loss: 0.8249 	Validation Accuracy: 0.606 	time: 111.8 	lr=0.01
Validation loss decreased from inf to 0.824876. Model was saved
Epoch: 2 	Training Loss: 0.7153 	Validation Loss: 0.6630 	Validation Accuracy: 0.714 	time: 111.3 	lr=0.01
Validation loss decreased from 0.824876 to 0.663032. Model was saved
Epoch: 3 	Training Loss: 0.6962 	Validation Loss: 0.6458 	Validation Accuracy: 0.723 	time: 111.3 	lr=0.01
Validation loss decreased from 0.663032 to 0.645784. Model was saved
Epoch: 4 	Training Loss: 0.6573 	Validation Loss: 0.6476 	Validation Accuracy: 0.716 	time: 111.3 	lr=0.01
Epoch: 5 	Training Loss: 0.6548 	Validation Loss: 0.6368 	Validation Accuracy: 0.724 	time: 111.3 	lr=0.01
Validation loss decreased from 0.645784 to 0.636848. Model was saved
Epoch: 6 	Training Loss: 0.6510 	Validation Loss: 0.6345 	Validation Accuracy: 0.711 	time: 111.1 	lr=0.01
Validation loss decreased from 0.636848 to 0.634543. Model was saved
Epoch: 7 	Training

# The following module can be run separately if trained weights are available

In [10]:
# MODEL_WEIGHTS_FILE = 'model_weights_v3_1__run_1.pt'

In [11]:
# import torch
# import torchvision.models as models
# import torch.nn as nn
# import torchvision.datasets
# import torchvision.transforms as transforms
# from PIL import ImageFile

# ImageFile.LOAD_TRUNCATED_IMAGES = True
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model_transfer = models.resnet152(pretrained=True).to(device)
    
# model_transfer.fc = nn.Sequential(
#                nn.Linear(2048, 128),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 3)).to(device)
# model_transfer.load_state_dict(torch.load(MODEL_WEIGHTS_FILE, map_location=device))

In [12]:
# TEST_DATA_PATH = '../data/test'
# transform_plain = transforms.Compose([
#                             transforms.Resize(224),
#                             transforms.CenterCrop(224),
#                             transforms.ToTensor(),
#                             transforms.Normalize(
#                                     mean=(0.485, 0.456, 0.406),
#                                     std =(0.229, 0.224, 0.225))
#                            ]) 

# if not 'data_transfer' in locals():
#     print("create empty data_transfer")
#     data_transfer = {}
# if not 'loaders_transfer' in locals():
#     print("create empty loaders_transfer")
#     loaders_transfer = {}
# data_transfer['test'] = torchvision.datasets.ImageFolder(TEST_DATA_PATH, transform=transform_plain)
# loaders_transfer['test'] = torch.utils.data.DataLoader(data_transfer['test'],
#                                           batch_size=1,
#                                           shuffle=False,
#                                           num_workers=4)

# import torch.optim as optim

# criterion_transfer = nn.CrossEntropyLoss()

In [13]:
import numpy as np
import time

def test(loaders, model, criterion):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()

    for batch_idx, (data, target) in enumerate(loaders['test']):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        # accumulate test loss 
        test_loss += loss.item()*data.size(0)
        
        # convert output probabilities to predicted class
        preds = output.data.max(1, keepdim=True)[1]
    
        # compare predictions to true label
        if torch.cuda.is_available():
            correct += torch.sum(preds == target.data)
        else:
            correct += np.sum(np.squeeze(preds.eq(target.data.view_as(preds))).cpu().numpy())
        total += data.size(0)
        
    test_loss = test_loss/len(loaders_transfer['test'].dataset)      
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function   
test_start = time.time()
test(loaders_transfer, model_transfer, criterion_transfer)
print("Total testing time: {:.2f} seconds".format(time.time() - test_start))

Test Loss: 0.701256


Test Accuracy: 71% (142/200)
Total testing time: 10.45 seconds
